In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install trl==0.15.2 peft unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [1]:
from unsloth.chat_templates import get_chat_template
import json
import torch
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Initialize model and tokenizer
model_name = "Qwen/Qwen2.5-0.5B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map="auto",
    trust_remote_code=True,
)

# Configure model for training
model.config.use_cache = False  # Disable KV caching for training

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
# max_seq_length = 2048
# dtype = None  # Auto-detect
# load_in_4bit = True

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "Qwen/Qwen2.5-0.5B",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16,
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                        "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0,
#     bias = "none",
#     use_gradient_checkpointing = "unsloth",
#     random_state = 3407,
# )

In [10]:
def prepare_20q_text_dataset(json_path, tokenizer):
    """
    Prepare 20 Questions dataset as plain text input-output pairs.
    """
    # Load the JSON data
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    train_examples = []
    
    for example in data:
        lines = example["lines"]
        target_word = example["word"][0] if example["word"] else ""
        correct = example["correct"]
        
        # Build conversation history
        history = []
        history.append({"role": "user", "content": "Let's play 20 Questions. You are playing the role of the Questioner. Please ask a yes-or-no question and try to guess the keyword."})
            
        # Process each conversation into training examples
        for i in range(0, len(lines)):
            sentence = lines[i].split("? ")
            if len(sentence) < 2:
                continue
            question = sentence[0] + "?"
            answer = sentence[1]
            
            # Add current question as assistant message
            history.append({"role": "assistant", "content": question})
            # Add current answer as user message
            if i+1 == len(lines):
                if correct:
                    history.append({"role": "user", "content": f"{answer}! You guess the right answer ({target_word})"})
                else:
                    history.append({"role": "user", "content": f"{answer}! You lose. Better luck next time!"})
            else:
                history.append({"role": "user", "content": answer})
        # Apply chat template
        train_example = {
            "text": tokenizer.apply_chat_template(
                history, 
                tokenize=False, 
                add_generation_prompt=False
            ) + tokenizer.eos_token
        }
        
        train_examples.append(train_example)
    
    # Convert to Dataset format
    return Dataset.from_list(train_examples)

# Create training dataset
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
train_dataset = prepare_20q_text_dataset("../input/raw-20q/train.json", tokenizer)
# train_dataset = train_dataset.train_test_split(test_size=0.1)
print(f"Created dataset with {len(train_dataset)} examples")

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False  # We're not doing masked language modeling
# )

Created dataset with 100000 examples


In [7]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = train_dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 3,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         # num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 1000,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none", # Use this for WandB etc
#     ),
# )

NameError: name 'max_seq_length' is not defined

In [13]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

# Define LoRA configuration for parameter-efficient fine-tuning
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj",],
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)

# Define training arguments
training_args = SFTConfig(
    output_dir="qwen-20q-sft",
    max_steps = 1000,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4, 
    warmup_steps = 5,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=200,
    save_total_limit=3,
    optim = "adamw_8bit",
    lr_scheduler_type = "linear",
    seed = 3407,
    fp16=True,
    report_to="none",
    dataset_num_proc = 3,
    max_seq_length=2048
)

# Create SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)

Tokenizing to ["text"] (num_proc=3):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [14]:
trainer.train()

Step,Training Loss
10,2.632000
20,1.902400
30,1.751100
40,1.728800
50,1.701100
60,1.701200
70,1.688900
80,1.672800
90,1.680200
100,1.669700


TrainOutput(global_step=1000, training_loss=1.6610012664794922, metrics={'train_runtime': 1000.165, 'train_samples_per_second': 7.999, 'train_steps_per_second': 1.0, 'total_flos': 8081271101583360.0, 'train_loss': 1.6610012664794922})

In [15]:
model.save_pretrained("sft_model")
tokenizer.save_pretrained("sft_model")

('sft_model/tokenizer_config.json',
 'sft_model/special_tokens_map.json',
 'sft_model/vocab.json',
 'sft_model/merges.txt',
 'sft_model/added_tokens.json',
 'sft_model/tokenizer.json')